In [1]:
import os 
import matplotlib
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import math
import re
import datetime
import joblib

from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = int(date.strftime("%Y"))

matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = "#949494"

%matplotlib inline

In [2]:
od.download('https://www.kaggle.com/c/playground-series-s4e9')
os.listdir('playground-series-s4e9')
train = pd.read_csv('playground-series-s4e9/train.csv', low_memory=False)
test = pd.read_csv('playground-series-s4e9/test.csv', low_memory=False)
sub = pd.read_csv('playground-series-s4e9/sample_submission.csv')

Skipping, found downloaded files in "./playground-series-s4e9" (use force=True to force download)


In [3]:
def convert_engine(val):
    d = {
        'engine_HP': np.nan,
        'engine_L': np.nan,
        'engine_cylinder': np.nan,
        'engine_cylinder_type': np.nan,
        'engine_fuel_type': np.nan,
        'engine_type': np.nan,
        'engine_v': np.nan,
        'engine_vtype': np.nan
    }
    spl = val.replace('-', '').split(' ')
    pos_v1 = -1
    pos_v2 = -1
    for i, v in enumerate(spl):
        if v.endswith('HP'):
            d['engine_HP'] = float(v[:-2])
        elif v.endswith('L'):
            d['engine_L'] = float(v[:-1])
        elif v == 'Engine' and spl[i-1] == 'Cylinder':
            d['engine_cylinder'] = abs(float(spl[i-2].replace('V', '')))
        elif v == 'Fuel':
            d['engine_fuel_type'] = spl[i - 1]
        elif v == 'Liter':
            d['engine_L'] = float(spl[i-1])
        elif v.startswith('V') or v.startswith('H') or v.startswith('I'):
            try:
                d['engine_cylinder'] = float(v[1:])
                d['engine_cylinder_type'] = v[0]
                pos_v1 = i
            except:
                pass
        elif v == 'DOHC' or v == 'OHV':
            d['engine_type'] = ' '.join(spl[i:])
            if pos_v2 != -1:
                d['engine_vtype'] = ' '.join(spl[pos_v2+1:i])
            elif pos_v1 != -1:
                d['engine_vtype'] = ' '.join(spl[pos_v1+1:i])
        elif v.endswith('V'):
            try:
                d['engine_v'] = float(v[:-1])
                pos_v2 = i
            except:
                pass
    return d

def test_params(ModelClass, **params):
    model = ModelClass(**params).fit(X_train, Y_train)
    train_rmse = rmse(model.predict(X_train), Y_train)
    val_rmse = rmse(model.predict(X_val), Y_val)
    return train_rmse, val_rmse


def test_param_and_plot(ModelClass, param_name, param_values, **other_params):
    train_errors, val_errors = [], [] 
    for value in param_values:
        params = dict(other_params)
        params[param_name] = value
        train_rmse, val_rmse = test_params(ModelClass, **params)
        train_errors.append(train_rmse)
        val_errors.append(val_rmse)
    
    plt.figure(figsize=(10,6))
    plt.title('Overfitting curve: ' + param_name)
    plt.plot(param_values, train_errors, 'b-o')
    plt.plot(param_values, val_errors, 'r-o')
    for x,y in zip(param_values,val_errors):
        label = "{:.2f}".format(y)
        plt.annotate(label,
                    (x,y),
                    textcoords="offset points",
                    xytext=(0,10),
                    ha='center')

    plt.xlabel(param_name)
    plt.ylabel('RMSE')
    plt.legend(['Training', 'Validation'])
    plt.show()

In [4]:
average_prices_by_brand = train.groupby('brand')['price'].mean()

train = train.fillna('dash')
train = train.replace('–', 'dash')

test = test.fillna('dash')
test = test.replace('–', 'dash')

accident_dict = {
    'None reported': 0,
    'At least 1 accident or damage reported': 1,
    'uknown': 2
}

clean_title_dict = {
    'Yes': 0,
    'uknown': 1
}

expensive_ext_color = ['Blue Caelum', 'Dark Sapphire', 'Bianco Monocerus', 'C / C', 'Ice',
       'Tempest', 'Beluga Black', 'Bianco Icarus Metallic', 
       'BLU ELEOS', 'Shadow Black', 'Nero Noctis', 'Sandstone Metallic',
       'Lizard Green', 'Balloon White', 'Onyx', 'Donington Grey Metallic',
       'China Blue', 'Diamond White', 'Rosso Corsa', 'Granite',
       'Rosso Mars Metallic', 'Carpathian Grey', 'Kemora Gray Metallic',
       'Grigio Nimbus', 'dash', 'Bianco Isis', 'Python Green', 'Fountain Blue',
       'Custom Color', 'Vega Blue', 'Designo Magno Matte',
       'Brands Hatch Gray Metallic', 'Rift Metallic', 'Gentian Blue Metallic',
       'Arancio Borealis', 'BLUE', 'Aventurine Green Metallic', 'Apex Blue',
       'Daytona Gray Pearl Effect', 'Daytona Gray Pearl Effect w/ Black Roof',
       'Matte White', 'Carpathian Grey Premium Metallic', 'Blue Metallic',
       'Santorini Black Metallic', 'Quartzite Grey Metallic',
       'Carrara White Metallic', 'BLACK', 'Kinetic Blue', 'Nero Daytona']

expensive_int_color = ['Dark Auburn', 'Hotspur', 'Cobalt Blue', 'Beluga Hide', 'Linen',
                       'Beluga', 'Black / Brown', 'Nero Ade', 'Sahara Tan', 'Portland']

inexpensive_ext_color = ['Gold', 'Frozen White', 'Beige', 'Volcanic Orange',
        'Mountain Air Metallic', 'Crystal Black Pearl', 'Crimson Red Tintcoat',
        'Snowflake White Pearl', 'Sonic Silver Metallic', 'Radiant Red Metallic II',
        'Remington Red Metallic', 'Gecko Pearlcoat', 'Dazzling White',
        'Blue Reflex Mica', 'White Platinum Tri-Coat Metallic', 'Purple',
        'Crystal White Pearl', 'Silver', 'Jet Black Mica', 'Red Quartz Tintcoat']

inexpensive_int_color =['Light Gray', 'Gray', 'Medium Pewter', 'Beige', 'Medium Ash Gray',
       'White / Brown', 'Ash', 'Deep Chestnut', 'Charcoal Black',
       'Whisper Beige', 'Gideon', 'Light Platinum / Jet Black', 'Sand Beige',
       'Mountain Brown', 'Ivory / Ebony', 'Black / Graphite', 'Canberra Beige',
       'Tupelo', 'Charcoal', 'Gold']

low_end_brand = average_prices_by_brand[average_prices_by_brand < 25000].index.tolist()
mid_range_brand = average_prices_by_brand[(average_prices_by_brand > 25000) & (average_prices_by_brand < 40000)].index.tolist()
high_end_brand = average_prices_by_brand[(average_prices_by_brand > 40000) & (average_prices_by_brand < 80000)].index.tolist()
luxury_brand = average_prices_by_brand[average_prices_by_brand > 80000].index.tolist()

def encode_columns(df):
    gases = df.fuel_type.value_counts().index
    for gas in gases:
        df.loc[(df['fuel_type'] == '–') & df['engine'].str.contains(gas), 'fuel_type'] = gas

    milage_per_year = df['milage'] / (year - df['model_year'])

    auto_transmission = df['transmission'].str.contains('A/T') + df['transmission'].str.contains(' At') + df['transmission'].str.contains('Automatic') + df['transmission'].str.contains('Dual Shift') + df['transmission'].str.contains('CVT Transmission') + df['transmission'].str.contains('CVT-F') + df['transmission'].str.contains('Overdrive Switch') + df['transmission'].str.contains('Variable')
    manual_transmission = df['transmission'].str.contains('M/T') + df['transmission'].str.contains(' Mt') + df['transmission'].str.contains('Manual')
    
    df['milage_per_year'] = milage_per_year.astype(int)
    
    df['auto_transmission'] = auto_transmission.astype(int)
    df['manual_transmission'] = manual_transmission.astype(int)

    df['turbo_engine'] = df['engine'].str.contains('Turbo').astype(int)
    df['supercharged_engine'] = df['engine'].str.contains('Supercharged').astype(int)
    df['dohc_engine'] = df['engine'].str.contains('DOHC').astype(int)
    df['ohv_engine'] = df['engine'].str.contains('OHV').astype(int)


    df['low_end_brand'] = df.brand.isin(low_end_brand).astype(int)
    df['mid_range_brand'] = df.brand.isin(mid_range_brand).astype(int)
    df['high_end_brand'] = df.brand.isin(high_end_brand).astype(int)
    df['luxury_brand'] = df.brand.isin(luxury_brand).astype(int)

    df['accident_encoded'] = df['accident'].map(accident_dict)
    df['clean_title_encoded'] = df['clean_title'].map(clean_title_dict)
    df['expensive_color_ext_encoded'] = df.ext_col.isin(expensive_ext_color).astype(int)
    df['expensive_color_int_encoded'] = df.int_col.isin(expensive_int_color).astype(int)
    df['inexpensive_color_ext_encoded'] = df.ext_col.isin(inexpensive_ext_color).astype(int)
    df['inexpensive_color_int_encoded'] = df.int_col.isin(inexpensive_int_color).astype(int)

    df['engine_fuel_type'].replace('dash', np.nan)
    df['engine_fuel_type'] = df.engine_fuel_type.fillna(df['fuel_type'])
    df['engine_fuel_type'] = df.engine_fuel_type.replace('Flexible', 'Flex').replace('E85 Flex Fuel', 'Flex')

    df['engine_HP_per_L'] = df['engine_HP']/df['engine_L']
    
    return df

train = pd.concat([train, pd.DataFrame.from_records(train['engine'].map(convert_engine).values)], axis = 1)
test = pd.concat([test, pd.DataFrame.from_records(test['engine'].map(convert_engine).values)], axis = 1)


train = encode_columns(train)
test = encode_columns(test)

In [5]:
input_cols = ['model_year', 'milage', 'milage_per_year', 'engine_fuel_type', 
       'low_end_brand', 'mid_range_brand', 'high_end_brand', 'luxury_brand', 
       'turbo_engine', 'dohc_engine', 'supercharged_engine', 'ohv_engine',
       'accident_encoded', 'clean_title_encoded', 
       'inexpensive_color_int_encoded', 'inexpensive_color_ext_encoded',
       'expensive_color_ext_encoded', 'expensive_color_int_encoded', 
       'auto_transmission', 'manual_transmission', 'engine_HP', 'engine_L', 'engine_HP_per_L',
       'engine_cylinder', 'engine_cylinder_type', 'engine_v', 'engine_vtype']

target_col = 'price'
target_col_list = ['price']

train_df, val_df = train_test_split(train, test_size=0.25, random_state=42)

In [6]:
'''
X_train = train_df[input_cols].copy()
Y_train = train_df[target_col].copy()

X_val = val_df[input_cols].copy()
Y_val = val_df[target_col].copy()

test_inputs = test[input_cols].copy()

numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train.select_dtypes('object').columns.tolist()
'''

"\nX_train = train_df[input_cols].copy()\nY_train = train_df[target_col].copy()\n\nX_val = val_df[input_cols].copy()\nY_val = val_df[target_col].copy()\n\ntest_inputs = test[input_cols].copy()\n\nnumeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()\ncategorical_cols = X_train.select_dtypes('object').columns.tolist()\n"

In [7]:
'''
imputer = SimpleImputer(strategy='mean').fit(X_train[numeric_cols])

X_train[numeric_cols] = imputer.transform(X_train[numeric_cols])
X_val[numeric_cols] = imputer.transform(X_val[numeric_cols])
test_inputs[numeric_cols] = imputer.transform(test_inputs[numeric_cols])

scaler = MinMaxScaler().fit(X_train[numeric_cols])

X_train[numeric_cols] = scaler.transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(X_train[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

X_train[encoded_cols] = encoder.transform(X_train[categorical_cols])
X_val[encoded_cols] = encoder.transform(X_val[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])

train_df = X_train[numeric_cols + encoded_cols]
val_df = X_val[numeric_cols + encoded_cols]
test = test_inputs[numeric_cols + encoded_cols]

numeric_cols_df = train_df.select_dtypes(include=np.number).columns.tolist()
categorical_cols_df = train_df.select_dtypes('object').columns.tolist()
'''

"\nimputer = SimpleImputer(strategy='mean').fit(X_train[numeric_cols])\n\nX_train[numeric_cols] = imputer.transform(X_train[numeric_cols])\nX_val[numeric_cols] = imputer.transform(X_val[numeric_cols])\ntest_inputs[numeric_cols] = imputer.transform(test_inputs[numeric_cols])\n\nscaler = MinMaxScaler().fit(X_train[numeric_cols])\n\nX_train[numeric_cols] = scaler.transform(X_train[numeric_cols])\nX_val[numeric_cols] = scaler.transform(X_val[numeric_cols])\ntest_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])\n\nencoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(X_train[categorical_cols])\nencoded_cols = list(encoder.get_feature_names_out(categorical_cols))\n\nX_train[encoded_cols] = encoder.transform(X_train[categorical_cols])\nX_val[encoded_cols] = encoder.transform(X_val[categorical_cols])\ntest_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])\n\ntrain_df = X_train[numeric_cols + encoded_cols]\nval_df = X_val[numeric_cols 

In [8]:
subsample_size = 10000
train_sample = train[input_cols+target_col_list].sample(n=subsample_size, random_state=42)


In [9]:
ag_root_mean_squared_error_scorer = make_scorer(name='root_mean_squared_error',
                                           score_func=rmse,
                                           optimum=0,
                                           greater_is_better=False)

In [7]:
autogluon_predictor  = TabularPredictor(label=target_col, eval_metric=ag_root_mean_squared_error_scorer).fit(train_data=train, presets='best', ag_args_fit={'num_gpus': 1})

NameError: name 'ag_root_mean_squared_error_scorer' is not defined

In [8]:
autogluon_predictor =  TabularPredictor.load("/home/kurios/Documents/used_car_price/Test/AutogluonModels/ag-20250612_104159")

/home/kurios/anaconda3/envs/rapids-25.04/lib/python3.12/site-packages/autogluon/common/utils/utils.py:97: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [9]:
train_preds = autogluon_predictor.predict(train)
test_preds = autogluon_predictor.predict(test)

In [10]:
print(train_preds)

0         10765.832031
1         11215.333984
2         13766.416016
3         63589.058594
4         80171.250000
              ...     
188528    44834.843750
188529    53386.667969
188530    87142.179688
188531    72824.445312
188532    30102.150391
Name: price, Length: 188533, dtype: float32


In [11]:
sub['price'] = test_preds
sub.to_csv('autogluon_submision.csv', index=None)

In [13]:
joblib.dump(autogluon_predictor, 'autogluon.pkl.pkl')

['autogluon.pkl.pkl']

In [ ]:
#autogluon_predictor.evaluate(train, silent=True)

{'root_mean_squared_error': -67426.6484375,
 'mean_squared_error': -4546353152.0,
 'mean_absolute_error': -18082.31640625,
 'r2': 0.268190860748291,
 'pearsonr': 0.5418868365371536,
 'median_absolute_error': np.float64(-8276.1015625)}

sample ALL DATA
preset Medium
'root_mean_squared_error': -74975.765625,
 'mean_squared_error': -5621365248.0,
 'mean_absolute_error': -19802.30859375,
 'r2': 0.09515029191970825,
 'pearsonr': 0.36414921355958385,
 'median_absolute_error': np.float64(-9149.26171875)}

Preset Best
Sample 10000
{'root_mean_squared_error': -73724.1484375,
 'mean_squared_error': -5435249664.0,
 'mean_absolute_error': -19925.84765625,
 'r2': 0.125108540058136,
 'pearsonr': 0.3590627359442765,
 'median_absolute_error': np.float64(-8298.19921875)}

W EXPENSIVE AND INEXPENSIVE COLOR
{'root_mean_squared_error': -73826.25,
 'mean_squared_error': -5450314752.0,
 'mean_absolute_error': -19591.48046875,
 'r2': 0.122683584690094,
 'pearsonr': 0.35641574353252903,
 'median_absolute_error': np.float64(-9263.404296875)}

WO INEXPENSIVE COLOR 
{'root_mean_squared_error': -76418.3828125,
 'mean_squared_error': -5839769600.0,
 'mean_absolute_error': -21874.41015625,
 'r2': 0.059994518756866455,
 'pearsonr': 0.28544079980672565,
 'median_absolute_error': np.float64(-9683.060546875)}

{'root_mean_squared_error': np.float64(-75892.00178206964),
 'mean_squared_error': -5759596032.0,
 'mean_absolute_error': -21492.87109375,
 'r2': 0.07289975881576538,
 'pearsonr': 0.2958477942083744,
 'median_absolute_error': np.float64(-9527.5)}

W INEXPENSIVE COLOR
WO EXPENSIVE COLOR
{'root_mean_squared_error': -74377.3203125,
 'mean_squared_error': -5531985920.0,
 'mean_absolute_error': -20652.509765625,
 'r2': 0.10953730344772339,
 'pearsonr': 0.33201927510226875,
 'median_absolute_error': np.float64(-9623.16015625)}

In [ ]:
#autogluon_predictor.leaderboard(train)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-73153.414062,-81877.140625,root_mean_squared_error,0.059603,0.004273,2.368062,0.059603,0.004273,2.368062,1,True,6
1,WeightedEnsemble_L2,-73724.148438,-77678.976562,root_mean_squared_error,3.212361,0.037848,148.643590,0.006116,0.000389,0.056708,2,True,12
2,NeuralNetFastAI,-73791.218750,-80215.140625,root_mean_squared_error,25.087382,0.010697,5.291494,25.087382,0.010697,5.291494,1,True,8
3,XGBoost,-74009.671875,-83761.046875,root_mean_squared_error,0.265538,0.006165,0.562571,0.265538,0.006165,0.562571,1,True,9
4,NeuralNetTorch,-74462.343750,-78759.960938,root_mean_squared_error,0.466911,0.008009,146.484321,0.466911,0.008009,146.484321,1,True,10
5,LightGBM,-74809.968750,-79008.328125,root_mean_squared_error,0.500258,0.004431,0.767570,0.500258,0.004431,0.767570,1,True,4
6,LightGBMLarge,-75694.914062,-85022.687500,root_mean_squared_error,0.174375,0.002395,1.233021,0.174375,0.002395,1.233021,1,True,11
7,LightGBMXT,-75826.484375,-78346.156250,root_mean_squared_error,1.890303,0.011552,1.326169,1.890303,0.011552,1.326169,1,True,3
8,ExtraTreesMSE,-77195.054688,-87994.976562,root_mean_squared_error,0.816726,0.082054,0.809238,0.816726,0.082054,0.809238,1,True,7
9,RandomForestMSE,-77821.289062,-93276.718750,root_mean_squared_error,0.876736,0.088144,1.457370,0.876736,0.088144,1.457370,1,True,5
